In [1]:
import os
import requests
import pandas as pd
import re
import asyncio
from datetime import datetime, timedelta
from prophet import Prophet
from duckduckgo_search import DDGS
from langchain_community.llms import Ollama
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from bs4 import BeautifulSoup

C:\Users\thegh\anaconda3\envs\llm_container_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [41]:
# 2. Forecast USD/EGP for tomorrow using Prophet
def forecast_usd(_: str = "") -> str:
    file_path = "usd_egp_historical.csv"
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found.")

    # Read without type coercion
    raw_df = pd.read_csv(file_path, dtype=str)

    if "Date" not in raw_df.columns or "Price" not in raw_df.columns:
        raise ValueError("CSV must contain 'Date' and 'Price' columns.")

    # Try automatic date parsing for mixed formats
    try:
        raw_df["ds"] = pd.to_datetime(raw_df["Date"])
    except Exception as e:
        raise ValueError(f"Failed to parse dates: {e}")

    # Convert Price to float
    raw_df["y"] = pd.to_numeric(raw_df["Price"], errors="coerce")

    # Clean and sort
    df = raw_df[["ds", "y"]].dropna()
    df = df.sort_values("ds")

    if len(df) < 2:
        raise ValueError("Not enough clean data to forecast. At least 2 rows are required.")

    # Prophet forecast
    model = Prophet(daily_seasonality=True)
    model.fit(df)

    future = model.make_future_dataframe(periods=2)
    forecast = model.predict(future)

    # Latest values
    today_date = df["ds"].iloc[-1].date()
    today_rate = df["y"].iloc[-1]

    tomorrow_row = forecast.iloc[-2]
    day_after_row = forecast.iloc[-1]

    return (
        f"Today ({today_date}):\n"
        f"1 USD = {today_rate:.2f} EGP\n\n"
        f"Tomorrow ({tomorrow_row['ds'].date()}):\n"
        f"1 USD = {tomorrow_row['yhat']:.2f} EGP\n\n"
        f"Day After Tomorrow ({day_after_row['ds'].date()}):\n"
        f"1 USD = {day_after_row['yhat']:.2f} EGP"
    )


In [42]:
# 3. Let LLM explain the forecast in natural language
def explain_forecast(_: str) -> str:
    llm = Ollama(model="mistral-openorca")
    prompt = (
        "The model used historical exchange rates (USD to EGP) and applied time series forecasting "
        "to predict tomorrow's rate. It considers the trend and daily seasonality to generate an expected value and range. "
        "Now explain this forecast simply for a general user."
    )
    return llm.invoke(prompt)

In [43]:
# 4. إعداد الأدوات و الـ Agent
llm = Ollama(model="mistral-openorca", temperature=0)

In [44]:
tools = [
    #Tool(name="Get USD Rate Today", func=get_usd_price, description="Scrapes today's USD to EGP exchange rate from the web."),
    Tool(name="Forecast USD Tomorrow", func=forecast_usd, description="Predicts the exchange rate for tomorrow using Prophet."),
    Tool(name="Explain Forecast", func=explain_forecast, description="Explains the forecast in simple terms using LLM.")
]

In [45]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [46]:
# 5. Run the agent
response = agent.run("Give me today's USD to EGP rate and a prediction for tomorrow with explanation.")
print("\n🤖 Agent Response:\n", response)



> Entering new AgentExecutor chain...


10:48:03 - cmdstanpy - INFO - Chain [1] start processing


 To find the current exchange rate, we need to use the Forecast USD Tomorrow action.

Action: Forecast USD Tomorrow
Action Input: Today's date

10:48:03 - cmdstanpy - INFO - Chain [1] done processing



Observation: Today (2025-06-30):
1 USD = 49.69 EGP

Tomorrow (2025-07-01):
1 USD = 49.20 EGP

Day After Tomorrow (2025-07-02):
1 USD = 49.10 EGP
Thought:

10:48:07 - cmdstanpy - INFO - Chain [1] start processing


 To predict the exchange rate for tomorrow, we need to use the Forecast USD Tomorrow action again with tomorrow's date as input.

Action: Forecast USD Tomorrow
Action Input: Tomorrow's date (2025-07-01)

10:48:07 - cmdstanpy - INFO - Chain [1] done processing



Observation: Today (2025-06-30):
1 USD = 49.69 EGP

Tomorrow (2025-07-01):
1 USD = 49.20 EGP

Day After Tomorrow (2025-07-02):
1 USD = 49.10 EGP
 To explain the forecast, we need to use the Explain Forecast action with the prediction for tomorrow as input.

Action: Explain Forecast
Action Input: Tomorrow's date (2025-07-01)
Observation:  The model is using information from past currency exchanges (like how many US Dollars were equivalent to Egyptian Pounds on different days) and trying to guess what tomorrow's rate might be. It looks at patterns, like if rates generally go up or down over time, and daily changes, like if rates usually change more during weekdays than weekends. This helps it make a best guess of what the rate will be tomorrow, along with a range that it thinks it could be in.
 Final Answer: Today's USD to EGP rate is 1 USD = 49.69 EGP. Tomorrow's forecast predicts that 1 USD will be equivalent to 49.20 EGP, and the day after tomorrow it might be 49.10 EGP. The model us